# Unsupervised Machine Learning: Market Basket Analysis

Tutorial: https://pythondata.com/market-basket-analysis-with-python-and-pandas/

Market basket analysis is the study of items that are purchased or grouped in a single transaction or multiple, sequential transactions. Understanding the relationships and the strength of those relationships is valuable information that can be used to make recommendations, cross-sell, up-sell, offer coupons, etc. It works by looking for combinations of items that occur together frequently in transactions.

In [2]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

## 1. Load Data

In [3]:
df = pd.read_excel('http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')

In [8]:
df.sample(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
367030,568831,22080,RIBBON REEL POLKADOTS,5,2011-09-29 11:35:00,1.65,16059.0,United Kingdom
319288,564846,22297,HEART IVORY TRELLIS SMALL,24,2011-08-30 15:16:00,1.25,14507.0,United Kingdom
284430,561871,22720,SET OF 3 CAKE TINS PANTRY DESIGN,3,2011-07-31 11:46:00,4.95,13018.0,United Kingdom
173678,551718,90003C,MIDNIGHT BLUE PAIR HEART HAIR SLIDE,1,2011-05-03 16:06:00,3.73,NaN,United Kingdom
322859,565234,85093,CANDY SPOT EGG WARMER HARE,2,2011-09-02 09:38:00,0.83,NaN,United Kingdom


## 2. Clean Data

In [9]:
# In this data, there are some invoices that are ‘credits’ instead of ‘debits’ 
# so we want to remove those. They are indentified with “C” in the InvoiceNo field. 

df = df[~df['InvoiceNo'].str.contains('C')]

## 3. Analysis

In [13]:
#We encode our data to show when a product is sold with another product. 
# If there is a zero, that means those products haven’t sold together. 

market_basket = df[df['Country'] =="United Kingdom"].groupby(
                ['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo')

In [15]:
market_basket.head()

Description,20713,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,...,wrongly coded 20713,wrongly coded 23343,wrongly coded-23343,wrongly marked,wrongly marked 23343,wrongly marked carton 22804,wrongly marked. 23343 in box,wrongly sold (22719) barcode,wrongly sold as sets,wrongly sold sets
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
# We want to convert all of our numbers to either a `1` or a `0` (negative numbers are 
# converted to zero, positive numbers are converted to 1).

def encode_data(datapoint):
    if datapoint <= 0: return 0
    if datapoint >= 1: return 1
    
market_basket = market_basket.applymap(encode_data)

The `apriori` function requires us to provide a minimum level of **support**. Support is defined as the percentage of time that an itemset appears in the dataset. If you set support = 50%, you’ll only get itemsets that appear 50% of the time. Setting the support level to high could lead to very few (or no) results and setting it too low could require an enormous amount of memory to process the data.

In [18]:
itemsets = apriori(market_basket, min_support=0.03, use_colnames=True)

In [20]:
itemsets.head()

,support,itemsets
0,0.045800,(6 RIBBONS RUSTIC CHARM)
1,0.031123,(60 CAKE CASES VINTAGE CHRISTMAS)
2,0.040336,(60 TEATIME FAIRY CAKE CASES)
3,0.046925,(ALARM CLOCK BAKELIKE GREEN)
4,0.035140,(ALARM CLOCK BAKELIKE PINK)


The final step is to build your **association rules** using the mxltend `association_rules` function. You can set the metric that you are most interested in (either `lift` or `confidence` and set the minimum threshold for the condfidence level (called `min_threshold`). The `min_threshold` can be thought of as the level of confidence percentage that you want to return. For example, if you set `min_threshold` to 1, you will only see rules with 100% confidence.

**Association analysis** uses a set of transactions to discover rules that indicate the likely occurrence of an item based on the occurrences of other items in the transaction.

In [21]:
rules = association_rules(itemsets, metric="lift", min_threshold=0.5)

In [22]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ALARM CLOCK BAKELIKE RED ),(ALARM CLOCK BAKELIKE GREEN),0.049818,0.046925,0.030159,0.605376,12.900874,0.027821,2.415149
1,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED ),0.046925,0.049818,0.030159,0.642694,12.900874,0.027821,2.659296
2,(GREEN REGENCY TEACUP AND SAUCER),(PINK REGENCY TEACUP AND SAUCER),0.050032,0.037658,0.030909,0.617773,16.404818,0.029024,2.517724
3,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.037658,0.050032,0.030909,0.820768,16.404818,0.029024,5.300218
4,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER ),0.050032,0.051264,0.037551,0.750535,14.640537,0.034986,3.803087
5,(ROSES REGENCY TEACUP AND SAUCER ),(GREEN REGENCY TEACUP AND SAUCER),0.051264,0.050032,0.037551,0.732497,14.640537,0.034986,3.551247
6,(JUMBO BAG RED RETROSPOT),(JUMBO BAG BAROQUE BLACK WHITE),0.103814,0.048747,0.030534,0.294118,6.033613,0.025473,1.347609
7,(JUMBO BAG BAROQUE BLACK WHITE),(JUMBO BAG RED RETROSPOT),0.048747,0.103814,0.030534,0.626374,6.033613,0.025473,2.398615
8,(JUMBO BAG PINK POLKADOT),(JUMBO BAG RED RETROSPOT),0.062085,0.103814,0.042051,0.677308,6.524245,0.035605,2.777218
9,(JUMBO BAG RED RETROSPOT),(JUMBO BAG PINK POLKADOT),0.103814,0.062085,0.042051,0.405057,6.524245,0.035605,1.576478
